In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(executable_path="chromedriver")

driver.get("https://comic.naver.com/index")

time.sleep(1)

webtoon_menu = driver.find_element(by=By.XPATH, value='//*[@id="menu"]/li[2]/a')
webtoon_menu.click()

webtoon_list = []
day_list = ["월", "화", "수", "목", "금", "토", "일"]

for day in day_list:
    dayOfTheWeek_menus = driver.find_elements(by=By.CSS_SELECTOR, value='.SubNavigationBar__link--PXX5B')
    for dayOfTheWeek_menu in dayOfTheWeek_menus:
        if(dayOfTheWeek_menu.text != day):
            continue

        dayOfTheWeek_menu.click()

        time.sleep(1)

        scrollTop = 0
        while True:
            driver.execute_script(f"window.scrollTo(0, {scrollTop});")

            scrollTop += 500
            time.sleep(0.5)
            if scrollTop > driver.execute_script("return document.body.scrollHeight"):
                break

        time.sleep(0.5)

        webtoon_items = driver.find_elements(by=By.CSS_SELECTOR, value='.ContentList__content_list--q5KXY .item')
        webtoon_items_list = []
        for item in webtoon_items:
            title = item.find_element(by=By.CSS_SELECTOR, value='.ContentTitle__title--e3qXt').text
            author = item.find_element(by=By.CSS_SELECTOR, value='.ContentAuthor__author--CTAAP').text
            rating = item.find_element(by=By.CSS_SELECTOR, value='.Rating__star_area--dFzsb > .text').text
            img = item.find_element(by=By.CSS_SELECTOR, value='.Poster__image--d9XTI').get_attribute(name="src")
            
            item_dict = {
                "title": title,
                "author": author,
                "rating": rating,
                "img": img
            }
            webtoon_items_list.append(item_dict)

        webtoon_by_day_dict = {
            "day": day,
            "webtoon_items_list": webtoon_items_list
        }

        webtoon_list.append(webtoon_by_day_dict)

df = pd.DataFrame(webtoon_list)
print(df)


  day                                 webtoon_items_list
0   월  [{'title': '참교육', 'author': '채용택 / 한가람', 'rati...
1   화  [{'title': '너는 그냥 개그만화나 그려라', 'author': '조석', ...
2   수  [{'title': '화산귀환', 'author': 'LICO / 비가', 'rat...
3   목  [{'title': '선천적 얼간이들', 'author': '가스파드', 'rati...
4   금  [{'title': '외모지상주의', 'author': '박태준', 'rating'...
5   토  [{'title': '호랑이형님', 'author': '이상규', 'rating':...
6   일  [{'title': '신혼일기', 'author': '자까', 'rating': '...


In [18]:
new_list = []

for webtoon in webtoon_list:
    for webtoon_item in webtoon.get("webtoon_items_list"):
        new_dict = {
            "day": webtoon.get("day")
        }
        new_dict.update(webtoon_item)
        new_list.append(new_dict)

df = pd.DataFrame(new_list)
print(df)
df.to_csv("webtoon.xlsx")

    day                 title                        author rating  \
0     월                   참교육                     채용택 / 한가람   9.87   
1     월               뷰티풀 군바리                      설이 / 윤성원   9.81   
2     월                  신의 탑                           SIU   9.88   
3     월                똑 닮은 딸                            이담   9.98   
4     월                  백수세끼                            치즈   9.85   
5     월               퀘스트지상주의                      박태준 만화회사   9.75   
6     월             장씨세가 호위무사                     김인호 / 조형근   9.93   
7     월                윈드브레이커                           조용석   9.83   
8     월                소녀의 세계                           모랑지   9.92   
9     월       버림받은 왕녀의 은밀한 침실                      혜니 / 성혜림   9.96   
10    월    신화급 귀속 아이템을 손에 넣었다                      정선율 / 헤스   9.90   
11    월                우아한 욕망                           김종건   9.91   
12    월                  별난식당                           HO9   9.96   
13    월             

In [2]:
import json

with open("webtton.json", "w", encoding="utf-8") as f:
    json.dump(webtoon_list, f, indent=4, ensure_ascii=False)